# Steckbrief einer elementaren Funktion

Zuerst wird in der gegebenen Funktion für alle 'c' eine zufällige Konstante aus der Menge {1,2,3} eingesetzt. Daher wird diese zunächst als String erwartet.

In [174]:
from sympy import *
import re
import random
x,c = symbols('x c')

f_str = 'c+c*(x**2-c)/x**4'

### Zufällig ausgewählte Funktion:

In [175]:
f = sympify(re.sub(r"c(?!o)", lambda m: str(random.choice([1,2,3])), f_str))
print(f)
f

1 + (2*x**2 - 2)/x**4


1 + (2*x**2 - 2)/x**4

### Definitionsbereich

In [176]:
domain = calculus.util.continuous_domain(f, x, S.Reals)
print(str(domain))
domain

Union(Interval.open(-oo, 0), Interval.open(0, oo))


Union(Interval.open(-oo, 0), Interval.open(0, oo))

### Wertebereich

**Achtung!** Der Wertebereich kann unter Umständen nicht berechenbar sein 

In [177]:
f_range = None
try:
    f_range = calculus.util.function_range(f, x, S.Reals)
    print(str(f_range))
except NotImplementedError:
    print('Wertebereich nicht berechenbar')
f_range

Interval(-oo, 3/2)


Interval(-oo, 3/2)

### Unstetigkeitsstellen

In [178]:
try:
    f_singularities = singularities(f, x, S.Reals)
    for s in f_singularities:
        left_limit = limit(f, x, s, '-')
        right_limit = limit(f, x, s, '+')
        if left_limit.is_real and left_limit == right_limit:
            print('Hebbare Lücke: ', s)
        elif left_limit.is_real and right_limit.is_real and left_limit != right_limit:
            print('Sprungstelle: ', s)
        elif any([not l for l in [left_limit, right_limit]]):
            print('Wesentliche Singularität: ', s)
        else:
            print('Polstelle: ', s)
except NotImplementedError:
    print('Unstetigkeitsstellen nicht berechenbar')

Polstelle:  0


### y-Achsenschnitt

In [179]:
y_intercept = f.subs(x, 0)
y_intercept

zoo

### Nullstellen

Eine Funktion kann eine endliche oder unendliche Menge an Nullstellen haben, daher muss hier differenziert werden.

Die Funktion `solveset()` liefert die genaue Menge der Nullstellen algebraisch definiert. Diese ist jedoch für spätere Filterung schlechter geeignet.

In [180]:
all_zeros = solveset(f, x, S.Reals)
infinite_zeros = not all_zeros.is_FiniteSet

if infinite_zeros:
    print('Menge der Nullstellen ist unendlich')
else:
    print('Menge der Nullstellen ist endlich')
all_zeros

Menge der Nullstellen ist endlich


{-sqrt(-1 + sqrt(3)), sqrt(-1 + sqrt(3))}

Mit Hilfe von `solve()` wird immer eine endliche Menge zurückgegeben, die allerdings im Falle unendlicher Nullstellen nicht vollständig ist (z.B. bei $sin(x)$):

In [181]:
finite_zeros = solve(f, x) if infinite_zeros else all_zeros
finite_zeros

{-sqrt(-1 + sqrt(3)), sqrt(-1 + sqrt(3))}

### Ableitung

In [182]:
fd = diff(f, x)
fd

4/x**3 - 4*(2*x**2 - 2)/x**5

### Extremstellen

In [187]:
fd_zeros = solveset(fd, x, S.Reals)
fdd = diff(fd, x)
extrema = []
if fd_zeros.is_empty:
    print('Keine Extremstellen')
elif fd_zeros.is_FiniteSet:
    for z in fd_zeros:
        val = fdd.subs(x, z)
        print(z, val)
        if val.is_negative:
            print('Maximum: ', z)
            extrema.append((z, 'max'))
        elif val.is_positive:
            print('Minimum: ', z)
            extrema.append((z, 'min'))
else:
    extrema = None
    print('Extremstellen nicht elementar berechenbar oder unendlich')
    print(fd_zeros)

sqrt(2) -2
Maximum:  sqrt(2)
-sqrt(2) -2
Maximum:  -sqrt(2)


### Wendestellen

In [185]:
fdd_zeros = solveset(fdd, x, S.Reals)
inflections = FiniteSet()
if fdd_zeros.is_empty:
    print('Keine Wendepunkte')
elif fdd_zeros.is_FiniteSet:
    fddd = diff(fd, x)
    for z in fdd_zeros:
        val = fddd.subs(x, z)
        if not val.is_zero:
            print('Wendepunkt: z')
            inflections += FiniteSet(z)
else:
    inflections = None
    print('Wendepunkte nicht elementar berechenbar oder unendlich')
    print(fdd_zeros)

### Monotonieintervalle

In [ ]:
# intervals = []
# last_val = -oo
# if extrema:
#     if len(extrema):
#         for n in FiniteSet(extrema.keys()):
#             interval = Interval(last_val, n)
#             intervals.append(interval, )